# Adding support for t-dependent Hamiltonians to `TrotterQRTE` (#9565)

### Why?
* `TrotterQRTE` should support parameterized Hamiltonians (`t_param`)
* [`SparsePauliOp` should support parameter assignments -- added in `#9796`]
* `aux_operators` should be measured at **every step** during Trotter evolution


### Trotter Evolution

* H can carry time-dependence
$$H = \sum_l a_l H_l$$

* Trotterized time-evolution operator
$$
U(t) = \exp \Bigl( -i H t \Bigr) = \Bigl( e^{ -i \frac{t}{n} \sum_l a_l H_l} \Bigr)^n$$
$$\longrightarrow \quad
U \approx \Bigl( \prod_{l=1}^L e^{-i a_l H_l \Delta t} \Bigr)^n
$$

### Trotter Evolution with t-dependent Hamiltonian

* H can carry time-dependence
$$H(t) = \sum_l a_l(t) H_l$$

* Trotterized time-evolution operator
$$
U(t_f,t_0) = \mathcal{T} \exp \Biggl( -i \int_{t_0}^{t_f} \mathrm{d}{t'} \, H(t') \Biggr)$$
$$\longrightarrow \quad 
U \approx \prod_{m=n}^1 e^{ -i \Delta t \sum_l a_l(m \Delta t) H_l}$$
$$\longrightarrow \quad
U \approx \prod_{m=n}^1 \prod_{l=1}^L e^{-i a_l(m \Delta t) H_l \Delta t}
$$

## Main changes:
#### Previously:
* initialize `TimeEvolutionProblem` and `TrotterQRTE` (and potentially `ProductFormula`)
* call `TrotterQRTE.evolve(TimeEvolutionProblem)`
    * `evolve()` then called `PauliEvolutionGate` which constructed **one gate for the entire Trotter evolution**
    * the `evolution_gate` was then used to measure observables **at the end of the full evolution**

#### Now:
* added argument `TrotterQRTE(..., num_timesteps, ...)` otherwise **interface is unchanged**
* initialize `TimeEvolutionProblem` and `TrotterQRTE` (and potentially `ProductFormula`)
* call `TrotterQRTE.evolve(TimeEvolutionProblem)`
    * `evolve()` loops over `num_timesteps`
    * at each step:
        * constructs `PauliEvolutionGate` from Hamiltonian with bound parameters
        * measures `aux_operators`

#### Careful
* must set `reps=1` in `ProductFormula(..., reps = 1, ...)` (see documentation of `TrotterQRTE`)

In [2]:
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.algorithms import TimeEvolutionProblem
from qiskit.algorithms.time_evolvers import TrotterQRTE
from qiskit.synthesis import LieTrotter

import numpy as np

In [3]:
A = lambda t: 1 - t
B = lambda t: t
t_param = Parameter("t")

op1 = SparsePauliOp(["X"], np.array([2*A(t_param)]))
op2 = SparsePauliOp(["Z"], np.array([2*B(t_param)]))
op3 = SparsePauliOp(["Y"], np.array([1000]))
H_t = op1 + op2 + op3
print(H_t)

aux_op = [SparsePauliOp(["Z"])]

T = 2
reps = 10

init = QuantumCircuit(1)
# init.h(0)

SparsePauliOp(['X', 'Z', 'Y'],
              coeffs=[ParameterExpression(2.0 - 2.0*t), ParameterExpression(2.0*t), (1000+0j)])


In [4]:
evolution_problem = TimeEvolutionProblem(
    H_t,
    time=T,
    initial_state=init,
    aux_operators=aux_op,
    t_param=t_param
)
pf = LieTrotter(reps=1)
estimator = Estimator()

trotter_qrte = TrotterQRTE(product_formula=pf, num_timesteps=reps, estimator=estimator)
result = trotter_qrte.evolve(evolution_problem)

In [5]:
full_evo_circ = result.evolved_state

print(full_evo_circ.decompose().decompose())

   ┌──────────┐┌──────────┐┌─────────┐┌──────────┐┌──────────┐┌─────────┐»
q: ┤ Rx(0.64) ├┤ Rz(0.16) ├┤ Ry(400) ├┤ Rx(0.48) ├┤ Rz(0.32) ├┤ Ry(400) ├»
   └──────────┘└──────────┘└─────────┘└──────────┘└──────────┘└─────────┘»
«   ┌──────────┐┌──────────┐┌─────────┐┌──────────┐┌──────────┐┌─────────┐»
«q: ┤ Rx(0.32) ├┤ Rz(0.48) ├┤ Ry(400) ├┤ Rx(0.16) ├┤ Rz(0.64) ├┤ Ry(400) ├»
«   └──────────┘└──────────┘└─────────┘└──────────┘└──────────┘└─────────┘»
«   ┌───────┐┌─────────┐┌─────────┐┌───────────┐┌──────────┐┌─────────┐»
«q: ┤ Rx(0) ├┤ Rz(0.8) ├┤ Ry(400) ├┤ Rx(-0.16) ├┤ Rz(0.96) ├┤ Ry(400) ├»
«   └───────┘└─────────┘└─────────┘└───────────┘└──────────┘└─────────┘»
«   ┌───────────┐┌──────────┐┌─────────┐┌───────────┐┌──────────┐┌─────────┐»
«q: ┤ Rx(-0.32) ├┤ Rz(1.12) ├┤ Ry(400) ├┤ Rx(-0.48) ├┤ Rz(1.28) ├┤ Ry(400) ├»
«   └───────────┘└──────────┘└─────────┘└───────────┘└──────────┘└─────────┘»
«   ┌───────────┐┌──────────┐┌─────────┐┌──────────┐┌─────────┐┌─────────┐
«q: ┤ Rx(-0.64) ├┤ 

In [6]:
aux_op_res = result.aux_ops_evaluated
obs_evo = result.observables

print(f"final result (previous only aux_op evaluation):  {aux_op_res}\n")
print("Newly added aux_op evaluations at each time step:")
for i, m in enumerate(obs_evo):
    print(f"step {i}:  <Z> = {m}")

final result (previous only aux_op evaluation):  [(0.5196318320730456, {})]

Newly added aux_op evaluations at each time step:
step 0:  <Z> = [(1.0, {})]
step 1:  <Z> = [(-0.3403780024970582, {})]
step 2:  <Z> = [(-0.1921446709473405, {})]
step 3:  <Z> = [(0.9817563369486734, {})]
step 4:  <Z> = [(-0.5124163987127632, {})]
step 5:  <Z> = [(-0.037579783878145645, {})]
step 6:  <Z> = [(0.7999948160086598, {})]
step 7:  <Z> = [(-0.8506385802705987, {})]
step 8:  <Z> = [(0.8247792619454901, {})]
step 9:  <Z> = [(-0.5484314496609569, {})]
step 10:  <Z> = [(0.5196318320730456, {})]


In [7]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright